In [ ]:
%run "/Users/iakubovskii/AppBooster/AB_testing/AB_classic.py"
revenue_android = pd.read_pickle("/Users/iakubovskii/AppBooster/AB_testing/Data/Quantum_AB_historic/ARPU/Android/final_df")
revenue_ios = pd.read_pickle("/Users/iakubovskii/AppBooster/AB_testing/Data/Quantum_AB_historic/ARPU/IOS/final_df")

In [ ]:
LevelUpRew_ios =  revenue_ios.loc[:, :, 'LevelUpRew']
means = LevelUpRew_ios.groupby(['group_test'])['total_id_revenue'].mean()
duration = LevelUpRew_ios['time_to_group'].max() - LevelUpRew_ios['time_to_group'].min()
all_users = Counter(LevelUpRew_ios.index.get_level_values('group_test'))
dict_counter = dict(Counter(LevelUpRew_ios.index.get_level_values('appsflyer_id')))
dict_counter_1more = {key: values for key,values in dict_counter.items() if values > 1}
variants = LevelUpRew_ios.drop("time_to_group", axis=1).unstack(level=1)
variants.columns = variants.columns.droplevel(0)

repeat_users = LevelUpRew_ios.loc[dict_counter_1more.keys()].pivot_table(index = 'appsflyer_id',
                                                         columns = 'group_test',
                                                         values = 'total_id_revenue').fillna(0)
describe_stat = variants.describe(percentiles = [0.75, 0.9, 0.95, 0.99])
zero_revenues = (LevelUpRew_ios.drop("time_to_group", axis=1).unstack(level=1)==0).sum(axis=0)
box_plot = px.box(LevelUpRew_ios.reset_index(), 
             x="group_test", 
             y="total_id_revenue",
             color="group_test")

# Hist for non zero and removing 95% quantile
hist = px.histogram(LevelUpRew_ios.query('total_id_revenue > 0 & total_id_revenue < ' + str(describe_stat.loc['95%'].mean())
                    ).reset_index(), 
                    x="total_id_revenue", color="group_test")
hist.update_layout(barmode='overlay')
hist.update_traces(opacity=0.9)
# box_plot.show()

In [ ]:
def delete(df):
    for col in df.columns:
        locals()['data_' + col] = df[col].dropna().values
    datas = " , ".join('data_' + col for col in df.columns)
    exec(f'''locals()['stat_kru'], locals()['p_val'] = kruskal({datas})''')
    return stat_kru
delete(variants)